#### Importing Modules

In [ ]:
import os
%matplotlib inline
import tensorflow as tf
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Conv2D, Dense, BatchNormalization, Dropout, MaxPooling2D, Flatten

#### Defining Train and Test Directory

In [ ]:
train_dir = '/content/data/train/'
test_dir = '/content/data/test/'
classes = len(os.listdir('/content/data_aug/train/'))

#### Loading Training and Testing data

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory(train_dir,
                                                batch_size=10,
                                                target_size=(48,48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=10,
                                                target_size=(48,48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')

training_set.class_indices

Found 35000 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

#### Defining Model Architecture

In [ ]:
emotor = tf.keras.models.Sequential()   

emotor.add(Conv2D(10, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48, 1)))
emotor.add(Conv2D(20, kernel_size=(3, 3), activation='relu', padding='same'))
emotor.add(BatchNormalization())
emotor.add(MaxPooling2D(2, 2))
emotor.add(Dropout(0.25))

emotor.add(Conv2D(40, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)))
emotor.add(Conv2D(80, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
emotor.add(BatchNormalization())
emotor.add(MaxPooling2D(pool_size=(2, 2)))
emotor.add(Dropout(0.25))

emotor.add(Conv2D(160, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.02)))
emotor.add(Conv2D(320, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.02)))
emotor.add(BatchNormalization())
emotor.add(MaxPooling2D(pool_size=(2, 2)))
emotor.add(Dropout(0.25))

emotor.add(Flatten())
emotor.add(Dense(1024, activation='relu'))
emotor.add(Dropout(0.5))

emotor.add(Dense(classes, activation='softmax'))

emotor.compile(optimizer=Adam(lr=0.0001, decay=1e-6), 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

emotor.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 48, 48, 10)        100       
                                                                 
 conv2d_7 (Conv2D)           (None, 48, 48, 20)        1820      
                                                                 
 batch_normalization_3 (Batc  (None, 48, 48, 20)       80        
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 24, 24, 20)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 24, 24, 20)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 24, 24, 40)       

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


#### Defining Checkpoint saves, Early-Stopping and Learning rate decay

In [ ]:
chk_path = 'emotor.h5'

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_accuracy')

earlystop = EarlyStopping(monitor='val_accuracy', 
                          min_delta=0, 
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              factor=0.2, 
                              patience=6, 
                              verbose=1, 
                              min_delta=0.0001)


callbacks = [checkpoint, reduce_lr]

#### Training the Model

In [13]:
steps_per_epoch = training_set.n // training_set.batch_size
validation_steps = test_set.n // test_set.batch_size

emotor.fit(x=training_set,
                  validation_data=test_set,
                  epochs=70,
                  callbacks=callbacks,
                  steps_per_epoch=steps_per_epoch,
                  validation_steps=validation_steps)

Epoch 1/70
3499/3500 [============================>.] - ETA: 0s - loss: 6.7774 - accuracy: 0.2508
Epoch 1: val_loss improved from inf to 4.28522, saving model to emotor.h5
3500/3500 [==============================] - 47s 13ms/step - loss: 6.7766 - accuracy: 0.2509 - val_loss: 4.2852 - val_accuracy: 0.3045 - lr: 1.0000e-04
Epoch 2/70
3496/3500 [============================>.] - ETA: 0s - loss: 3.0163 - accuracy: 0.3417
Epoch 2: val_loss improved from 4.28522 to 2.31480, saving model to emotor.h5
3500/3500 [==============================] - 45s 13ms/step - loss: 3.0155 - accuracy: 0.3417 - val_loss: 2.3148 - val_accuracy: 0.3796 - lr: 1.0000e-04
Epoch 3/70
3498/3500 [============================>.] - ETA: 0s - loss: 2.0675 - accuracy: 0.3958
Epoch 3: val_loss improved from 2.31480 to 2.05579, saving model to emotor.h5
3500/3500 [==============================] - 45s 13ms/step - loss: 2.0673 - accuracy: 0.3959 - val_loss: 2.0558 - val_accuracy: 0.3662 - lr: 1.0000e-04
Epoch 4/70
3497/3500